In [10]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle

# Import Feature Engineered Sales Transaction file
sales_df = pd.read_csv('Sales-Transactions-Edited.csv')


# Top Selling Items

In [11]:
# Find the no of units sold of each product
# Find the unit price of each product (max of price considered, may required to be changed to median or mean)
top_sell_items_df = sales_df.groupby('Product').agg({'Qty':'sum', 'Rate':'max'})

# Reset the index by converting the Product into a column
top_sell_items_df.reset_index(inplace=True)


# Rank the product by most Qty sold
top_sell_items_df['Top_Sell_Rank'] = top_sell_items_df['Qty'].rank(method='min',ascending=False).astype(int)


# List the top 20 items sold
top_sell_items_df.sort_values('Qty',ascending=False).head(20)


,Product,Qty,Rate,Top_Sell_Rank
91,14 GREEN,3474510,400.0,1
282,CYCLE-BLU-10*12,741675,7.1,2
177,500ML PLASTIC BOULS,587600,3.8,3
314,DURGA 10*12 BLUE,530384,110.0,4
43,10*12 SARAS-NAT,433630,81.0,5
279,CYCLE-BK-10*12,431060,13.0,6
842,VISHNU 250ML,400239,35.0,7
638,ROBO 10*12,353880,5.5,8
37,10*12 KRISHNA-BK(10,349720,15.0,9
824,TIRUMALA-50(13*16),346776,28.0,10


# Most Popular Items

In [12]:
# Considered Date column instead of Voucher, in counting the no of orders placed for a product.
# This ignores the multiple no of orders created in a single day.
# Here the understanding is that, this being a wholesale business,
#      a customer places a 2nd order of the same product in a day, only when he/she notices a wrong qty placed on the order.
# If the business considers to have Voucher column, instead of Date column, all the Date column below needs to be replaced.


# Remove duplicate records at Product, Date and Party level
unique_order_items_df = sales_df.drop_duplicates(['Product','Date','Party'])


# Find the no of orders placed and the unique no of customers placed orders, of each product
most_popular_items_df = unique_order_items_df.groupby('Product').agg({'Date':'count', 'Party':'nunique'})
most_popular_items_df.columns=['No_of_Orders','No_of_Customers']

# Reset the index by converting the Product into a column
most_popular_items_df.reset_index(inplace=True)


# Products with high no of orders can be considered as most frequently purchased items
# To find the most popular items, include the no of customers purchased and provide more weightage to products purchased by more customers

# Weighted No_of_Orders (W) = O * (C / M)
# O = No_of_Orders
# C = No_of_Customers purchased the product
# M = Maximum no of customers made transactions in the entire period

O = most_popular_items_df['No_of_Orders']
C = most_popular_items_df['No_of_Customers']
M = most_popular_items_df['No_of_Customers'].max()

most_popular_items_df['Weighted_No_of_Orders'] = O * (C / M)

# Rank the product by weighted no of orders
most_popular_items_df['Popularity_Rank'] = most_popular_items_df['Weighted_No_of_Orders'].rank(method='min',ascending=False).astype(int)


# List of top 20 most popular items sold
most_popular_items_df.sort_values('Popularity_Rank',ascending=True).head(20)

,Product,No_of_Orders,No_of_Customers,Weighted_No_of_Orders,Popularity_Rank
91,14 GREEN,2365,244,2365.000000,1
825,TIRUMALA-50(16*20),2123,224,1948.983607,2
238,BLACK DOG-350ML,1945,206,1642.090164,3
223,APPLE WATER,1853,203,1541.635246,4
843,VISHNU 300ML,2060,160,1350.819672,5
842,VISHNU 250ML,1962,166,1334.803279,6
19,10*10 TEJA,1454,184,1096.459016,7
230,BAHUBALI WINE,1468,181,1088.967213,8
824,TIRUMALA-50(13*16),1491,176,1075.475410,9
465,NO-1,1178,195,941.434426,10


# Merge all the Ranks

In [13]:
# Merge Top Selling Items Rank and Popularity Rank dataframes
product_rankings_df = pd.merge(top_sell_items_df,most_popular_items_df,how='inner',on='Product')

# Get only the Product, Price and Rank columns
product_rankings_df = product_rankings_df[['Product','Rate','Top_Sell_Rank','Popularity_Rank']]

# List the Product Rankings
product_rankings_df.sort_values('Popularity_Rank',ascending=True).head(20)

,Product,Rate,Top_Sell_Rank,Popularity_Rank
91,14 GREEN,400.0,1,1
825,TIRUMALA-50(16*20),30.0,19,2
238,BLACK DOG-350ML,33.0,15,3
223,APPLE WATER,29.0,11,4
843,VISHNU 300ML,50.0,16,5
842,VISHNU 250ML,35.0,7,6
19,10*10 TEJA,142.0,64,7
230,BAHUBALI WINE,31.0,20,8
824,TIRUMALA-50(13*16),28.0,10,9
465,NO-1,160.0,123,10


# Write the Product Rankings into a .csv file

In [14]:
product_rankings_df.to_csv('Product-Rankings.csv',index=False)

# Create a Pickle (.pkl) file with the Ranking dataframe

In [15]:
pickle.dump(product_rankings_df, open('prod_ranking_model.pkl','wb'))